In [3]:
import pandas as pd
import numpy as np

In [4]:
# Import necessary libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from pytorch_tabnet.tab_model import TabNetRegressor


# Define the VAE model
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim * 2),  # latent_dim * 2 for mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid(),
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = torch.chunk(h, 2, dim=-1)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar


# Loss function for VAE
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction="sum")
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD


# Train the VAE
def train_vae(model, dataloader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for x, _ in dataloader:
            optimizer.zero_grad()
            recon_x, mu, logvar = model(x)
            loss = vae_loss(recon_x, x, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader.dataset)}")


# Function to encode data using trained VAE
def encode_data(model, dataloader):
    model.eval()
    encoded_data = []
    with torch.no_grad():
        for x, _ in dataloader:
            h = model.encoder(x)
            mu, _ = torch.chunk(h, 2, dim=-1)
            encoded_data.append(mu)
    return torch.cat(encoded_data)


# Load and preprocess your data
# Assuming X_train, y_train, X_test, y_test are your data arrays
# Normalize data, split into train/test, etc.

# Create DataLoader
batch_size = 32
train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define model, optimizer and train VAE
input_dim = X_train.shape[1]
latent_dim = 10
vae = VAE(input_dim, latent_dim)
optimizer = optim.Adam(vae.parameters(), lr=0.001)
train_vae(vae, train_loader, optimizer, epochs=50)

# Encode data using the trained VAE
encoded_X_train = encode_data(vae, train_loader).numpy()

# Train TabNet on the encoded data
tabnet = TabNetRegressor()
tabnet.fit(encoded_X_train, y_train, eval_set=[(encoded_X_train, y_train)])

# Evaluate TabNet on test data
# Encode test data using the trained VAE
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
encoded_X_test = encode_data(vae, test_loader).numpy()

# Predict and evaluate
preds = tabnet.predict(encoded_X_test)
# Evaluate your predictions (e.g., calculate RMSE, MAE, etc.)

ModuleNotFoundError: No module named 'torch'